### Library Import

In [ ]:
import os
from typing import Any, List, Dict, Tuple
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from xgboost import XGBRegressor
import lightgbm as lgb
import optuna
from sklearn.ensemble import VotingRegressor
from data_preprocessing import *
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

### Data Load

In [2]:
# 파일 호출
data_path: str = "/data/ephemeral/home/BTC/data"
train_df: pd.DataFrame = pd.read_csv(os.path.join(data_path, "train.csv")).assign(_type="train") # train 에는 _type = train 
test_df: pd.DataFrame = pd.read_csv(os.path.join(data_path, "test.csv")).assign(_type="test") # test 에는 _type = test
submission_df: pd.DataFrame = pd.read_csv(os.path.join(data_path, "test.csv")) # ID, target 열만 가진 데이터 미리 호출
df: pd.DataFrame = pd.concat([train_df, test_df], axis=0)

In [ ]:
# HOURLY_ 로 시작하는 .csv 파일 이름을 file_names 에 할딩
file_names: List[str] = [
    f for f in os.listdir(data_path) if f.startswith("HOURLY_") and f.endswith(".csv")
]

# 파일명 : 데이터프레임으로 딕셔너리 형태로 저장
file_dict: Dict[str, pd.DataFrame] = {
    f.replace(".csv", ""): pd.read_csv(os.path.join(data_path, f)) for f in file_names
}

for _file_name, _df in tqdm(file_dict.items()):
    # 열 이름 중복 방지를 위해 {_file_name.lower()}_{col.lower()}로 변경, datetime 열을 ID로 변경
    _rename_rule = {
        col: f"{_file_name.lower()}_{col.lower()}" if col != "datetime" else "ID"
        for col in _df.columns
    }
    _df = _df.rename(_rename_rule, axis=1)
    df = df.merge(_df, on="ID", how="left")


### Feature engineering

In [ ]:
# 모델에 사용할 컬럼, 컬럼의 rename rule을 미리 할당함
cols_dict: Dict[str, str] = {
    "ID": "ID",
    "target": "target",
    "_type": "_type",
    "hourly_market-data_liquidations_all_exchange_all_symbol_long_liquidations": "long_liquidations",
    "hourly_market-data_liquidations_all_exchange_all_symbol_short_liquidations": "short_liquidations",
    "hourly_market-data_open-interest_all_exchange_all_symbol_open_interest": "open_interest",
    "hourly_market-data_price-ohlcv_all_exchange_spot_btc_usd_close": "close",
    "hourly_market-data_taker-buy-sell-stats_all_exchange_taker_buy_volume": "taker_buy_volume",
    "hourly_market-data_taker-buy-sell-stats_all_exchange_taker_sell_volume": "taker_sell_volume",
    "hourly_network-data_addresses-count_addresses_count_active": "addresses_count_active",
    "hourly_network-data_addresses-count_addresses_count_sender": "addresses_count_sender",
    "hourly_network-data_addresses-count_addresses_count_receiver": "addresses_count_receiver",
    "hourly_network-data_block-bytes_block_bytes": "block_bytes",
    "hourly_network-data_block-interval_block_interval": "block_interval",
    "hourly_network-data_blockreward_blockreward": "blockreward",
    "hourly_network-data_difficulty_difficulty": "difficulty",
    "hourly_network-data_hashrate_hashrate": "hashrate",
    "hourly_network-data_supply_supply_total": "supply_total",
    "hourly_network-data_tokens-transferred_tokens_transferred_total": "tokens_transferred_total",
    "hourly_network-data_transactions-count_transactions_count_total": "transactions_count_total",
    "hourly_network-data_utxo-count_utxo_count": "utxo_count",
    "hourly_network-data_velocity_velocity_supply_total": "velocity_supply_total"
}
df = df[cols_dict.keys()].rename(cols_dict, axis=1)
df.shape

In [5]:
# continuous 열을 따로 할당해둠
conti_cols: List[str] = [
    "close",
    "open_interest",
    "difficulty",
    "supply_total",
    "utxo_count"
]

# 최대 24시간의 shift 피쳐를 계산
shift_list = shift_feature(
    df=df, conti_cols=conti_cols, intervals=[_ for _ in range(1, 6)]
)

# concat 하여 df 에 할당
df = pd.concat([df, pd.concat(shift_list, axis=1)], axis=1)

In [6]:
# _type에 따라 train, test 분리
train_df = df.loc[df["_type"]=="train"].drop(columns=["_type"])
test_df = df.loc[df["_type"]=="test"].drop(columns=["_type"])

### Model Train

In [7]:
def close_to_class(series: pd.Series) -> pd.Series:
    """close 변수를 target값으로 변환하는 함수입니다.

    Args:
        series (pd.Series): 변환을 원하는 close 변수

    Returns:
        pd.Series: 변환된 target 값
    """
    close = pd.DataFrame()
    close['close'] = series
    close['close_lag1'] = close['close'].shift(1)
    close['close_lag1_percent'] = (close['close'] - close['close_lag1']) / close['close_lag1']
    close['class'] = close['close']
    for i in range(close.shape[0]):
        if close.loc[i, 'close_lag1_percent'] < -0.005:
            close.loc[i, 'class'] = 0
        elif close.loc[i, 'close_lag1_percent'] < 0:
            close.loc[i, 'class'] = 1
        elif close.loc[i, 'close_lag1_percent'] < 0.005:
            close.loc[i, 'class'] = 2
        else:
            close.loc[i, 'class'] = 3
            
    return close["class"].shift(-1).fillna(method="ffill")

In [8]:
# 모델 평가
def evaluate(valid_target: pd.Series, 
             y_valid: pd.Series, 
             y_pred: np.ndarray, 
             metric: str
) -> float:
    """평가지표 metric을 반환하는 함수입니다.

    Args:
        valid_target: (pd.Series): k-fold로 분할한 target의 검증 데이터
        y_valid (pd.Series): k-fold로 분할한 close의 검증 데이터
        y_pred (np.ndarray): 모델을 사용하여 예측한 변수
        metric (str): 사용할 평가지표 metric 이름

    Returns:
        float: 사용할 평가지표 metric 값
    """
    if metric == "accuracy":
        y_pred_class = close_to_class(y_pred)
        return accuracy_score(valid_target, y_pred_class)
    elif metric == "mae":
        return mean_absolute_error(y_valid, y_pred)
    elif metric == "mse":
        return mean_squared_error(y_valid, y_pred)
    elif metric == "mape":
        return mean_absolute_percentage_error(y_valid, y_pred)

In [9]:
def model_train(
    model,
    X_train,
    y_train,
    cv: int = 5,
    metric: str = "accuracy"
) -> tuple[None, float]:
    """모델을 불러와 cross validation을 이용해 예측 성능을 확인하는 함수입니다.

    Args:
        model (Any): 학습을 수행할 모델
        X_train (Any): 훈련 데이터 피처
        y_train (Any): 훈련 데이터
        cv (int): KFold 검정 횟수
        metric (str): 성능 지표. accuracy(target), mae, mape mse 중 선택.

    Returns:
        _type_: _description_
    """
    kfold = KFold(n_splits=cv)
    score_list = []
    fold_model = []
    # warm_start는 모델의 속성으로, 같은 모델을 반복 학습할 때 이전 학습에서 학습된 파라미터를 초기화하지 않고 이어서 학습을 진행하는 옵션
    if hasattr(model, "warm_start"):
        model.warm_start = True
    
    for train_index, valid_index in kfold.split(X_train):
        X_train_fold, y_train_fold = X_train.iloc[train_index], y_train.iloc[train_index]
        X_valid, y_valid = X_train.iloc[valid_index], y_train.iloc[valid_index]

        valid_target = X_valid["target"]
        X_train_fold.drop("target", axis=1, inplace=True)
        X_valid.drop("target", axis=1, inplace=True)

        # preprocessing
        X_train_fold.fillna(X_train_fold.mean(), inplace=True)
        y_train_fold.fillna(y_train_fold.mean(), inplace=True)
        X_valid.fillna(X_valid.mean(), inplace=True)
        y_valid.fillna(y_valid.mean(), inplace=True)

        model.fit(X_train_fold, y_train_fold)
        fold_model.append(model)

        y_pred = model.predict(X_valid)
        score = evaluate(valid_target, y_valid, y_pred, metric=metric)
        score_list.append(score)
    
    return fold_model[np.argmax(score_list)], np.max(score_list)

In [10]:
X_train = train_df.drop(["ID", "close"], axis=1)
y_train = train_df["close"]
X_test = test_df.drop(["ID", "close"], axis=1)
y_test = test_df["close"]
X_test.fillna(X_test.mean(), inplace=True)
y_test.fillna(y_test.mean(), inplace=True)

### Model Ensemble

In [11]:
xgb_params = {
    'n_estimators': 159,
    'learning_rate': 0.044545369253400344,
    'max_depth': 7,
    'min_child_weight': 5,
    'colsample_bytree': 0.5878501242431816,
    'subsample': 0.7729401646786744,
    "booster": "gbtree",
    "device": "gpu",
    "random_state": 42
}
lgb_params = {
    "verbose" : -1,
    "num_leaves" : 55,
    "learning_rate" : 0.024012082225330366,
    "n_estimators": 245,
    "max_depth" : 4,
    "min_child_weight" : 2,
    "subsample" : 0.8121704803110622,
    "colsample_bytree" : 0.5396346553816902,
    "random_state" : 42
}

In [12]:
# 앙상블에 사용할 회귀 모델들
xgb_model = XGBRegressor(**xgb_params)
lgb_model = lgb.LGBMRegressor(**lgb_params)
lr_model = Lasso(alpha=20, max_iter=2000, random_state=42)

In [13]:
# Voting Regressor로 앙상블 진행
ensemble_model = VotingRegressor([('xgb', xgb_model), ('lgb', lgb_model), ('lr', lr_model)])

In [ ]:
# 학습
Best_ensemble_model, model_accuracy = model_train(ensemble_model, X_train, y_train, cv=5, metric="accuracy")
print(f"Ensemble model accuracy: {model_accuracy}")

### Inference

In [15]:
# lr predict
X_train_drop = fill_feature(X_train.drop("target", axis=1), method="mean")
Best_ensemble_model.fit(X_train_drop, y_train)

y_test_pred = Best_ensemble_model.predict(X_test.drop("target", axis=1))
y_test_pred_class = close_to_class(y_test_pred)

In [16]:
# lr output
submission_df = submission_df.assign(target = y_test_pred_class)
submission_df["target"] = submission_df["target"].astype(np.int8)
submission_df.to_csv("output.csv", index=False)

In [ ]:
submission_df['target'].value_counts()